In [ ]:
!gdown 1mU--DNhy8pWMTljj7jI3FvJwRYRHwAq5
!gdown 1yt9RDch0ZT9gtTfEBQLFFhGPw3PSOEh0
!gdown 13XpLuVuxI6ekdEf5UElKH_IMWrK8wZU1

## 1. Imports

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2. Data

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

imsize = 256

img_transforms = transforms.Compose([
    transforms.Resize((imsize, imsize)),
    transforms.ToTensor()
])


In [ ]:
def image_loader(image_name):
    image = Image.open(image_name)
    image = img_transforms(image).unsqueeze(0) # add new dim for batch size (index 0)
    return image.to(device, torch.float)

style_img1 = image_loader("style_img.jpg")
style_img2 = image_loader("style_img2.jpg")
content_img = image_loader("content_img.jpg")

In [ ]:
unloader = transforms.ToPILImage() # to use imshow

def imshow(tensor, title=None):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


plt.figure()
imshow(style_img1, title='Style Image1')

plt.figure()
imshow(style_img2, title='Style Image2')

plt.figure()
imshow(content_img, title='Content Image')

## 3. Loss Functions

### 3.1 Content Loss

In [ ]:
content_weight = 1
ContentLoss = nn.MSELoss()

### 3.2 Style Loss

In [ ]:
def gram_matrix(tensor):
    """
    Calculate the Gram Matrix of a given 4D tensor of shape [B, C, H, W].
    This is commonly used in style transfer to capture correlations between feature maps (style representation)

    Returns:
        torch.Tensor: The normalized Gram matrix of shape [B*C, B*C]
    """
    B, C, H, W = tensor.size()
    tensor = tensor.view(B * C, H * W)
    G = torch.mm(tensor, tensor.t())
    return G.div(B * C * H * W)

style_weight = 1e6
StyleLoss = nn.MSELoss()

## 3. Model

In [ ]:
from torchvision.models import vgg19, VGG19_Weights

VGG19_pretrained = vgg19(weights=VGG19_Weights.DEFAULT).features.eval()
VGG19_pretrained.to(device)

In [ ]:
class Normalization(nn.Module):
    def __init__(self):
        super(Normalization, self).__init__()
        self.mean = torch.tensor(torch.tensor([0.485, 0.456, 0.406]).to(device)).view(-1, 1, 1)
        self.std = torch.tensor(torch.tensor([0.229, 0.224, 0.225]).to(device)).view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

normalization = Normalization().to(device)

In [ ]:
content_layers = ['conv_4']
style_layers = ['conv_1', 'conv_2',
                'conv_3', 'conv_4',
                'conv_5']

In [ ]:
def get_features(pretrained_model, image):
    layers = {
        '0': 'conv_1',
        '5': 'conv_2',
        '10': 'conv_3',
        '19': 'conv_4',
        '28': 'conv_5'
    }
    features = {}
    x = image
    x = normalization(x)
    for name, pretrained_layer in pretrained_model._modules.items():
        x = pretrained_layer(x)
        if name in layers:
            features[layers[name]] = x
    return features


In [ ]:
def get_dual_style(style_features1, style_features2, style_layers):
    final_style_features = {}
    for layer in style_layers:
        sf1 = style_features1[layer]
        sf2 = style_features2[layer]
        ##############################YOUR CODE HERE############################
        # 1. Calculate the size of the first portion of sf1 to be used.
        #    This is a quarter of the number of channels (dimension 1).
        # 2. Concatenate the first portion of sf1 with the second portion of sf2
        # along the channel dimension.
        #    - The first portion of sf1 should contain the first 'sf1_size' channels.
        #    - The second portion of sf2 should contain the channels starting
        #      from 'sf1_size' to the end.
        ########################################################################
        sf1_size = sf1.size(1) // 4
        sf2_size = sf1.size(1) - sf1_size
        sf1 = sf1[:, :sf1_size, :, :]
        sf2 = sf2[:, sf1_size:, :, :]
        sf = torch.cat((sf1, sf2), dim=1)
        final_style_features[layer] = sf
    return final_style_features

In [ ]:
content_features = get_features(VGG19_pretrained, content_img)
style_features1 = get_features(VGG19_pretrained, style_img1)
style_features2 = get_features(VGG19_pretrained, style_img2)
final_style_features = get_dual_style(style_features1, style_features2, style_layers)

## 4. Training

In [ ]:
import torch.optim as optim

target_img = content_img.clone().requires_grad_(True).to(device)
optimizer = optim.Adam([target_img], lr=0.02)


In [ ]:
def style_tranfer_(model, optimizer, target_img,
                   content_features, style_features,
                   style_layers, content_weight, style_weight):

    optimizer.zero_grad()
    with torch.no_grad():
        target_img.clamp_(0, 1)
    target_features = get_features(model, target_img)

    content_loss = ContentLoss(content_features['conv_4'],
                               target_features['conv_4'])

    style_loss = 0
    for layer in style_layers:
        target_gram = gram_matrix(target_features[layer])
        style_gram = gram_matrix(style_features[layer])
        style_loss += StyleLoss(style_gram, target_gram)

    total_loss = content_loss*content_weight + style_loss*style_weight
    total_loss.backward(retain_graph=True)
    optimizer.step()
    return total_loss, content_loss, style_loss

In [ ]:
STEPS = 500

for step in range(STEPS):
    optimizer.zero_grad()
    with torch.no_grad():
        target_img.clamp_(0, 1)

    total_loss, content_loss, style_loss = style_tranfer_(VGG19_pretrained, optimizer, target_img,
                                                           content_features, final_style_features,
                                                           style_layers, content_weight, style_weight)
    if step % 100 == 99:
        print(f"Epoch [{step+1}/{STEPS}] Total loss: {total_loss.item():.6f} - \
                Content loss: {content_loss.item():.6f} - Style loss: {style_loss.item():.6f}")

    with torch.no_grad():
        target_img.clamp_(0, 1)

In [ ]:
plt.figure()
imshow(target_img.detach(), title='Output Image')
plt.show()